# Imports

In [1]:
import arxiv
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from fastapi import FastAPI
from pydantic import BaseModel
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv



## Constants

In [2]:
TOP_K_RESULTS = 1
DOCUMENT_CONTENT_CHARS_MAX = 400
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

## Load environment variables from my .env file

In [3]:
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY', 'nao encontrada')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'nao encontraaada')



##  Using API for wikipedia content - this will later be implemented in the "tools" for the agent


In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=TOP_K_RESULTS,document_content_chars_max=DOCUMENT_CONTENT_CHARS_MAX)   
wiki = WikipediaQueryRun(api_wrapper = api_wrapper)


## Loading the content from the web and splitting the text into chunks to be used in the FAISS
## PDF loader and splitting the text into chunks to be used in the FAISS

In [5]:
def load_and_split(loader, splitter, embeddings):
    docs = loader.load()
    documents = splitter.split_documents(docs)
    db = FAISS.from_documents(documents, embeddings)
    return db


In [6]:
web_db = load_and_split(WebBaseLoader("https://www.studypool.com/studyGuides/Alice_in_Wonderland/Characters"), 
                          RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP), 
                          OpenAIEmbeddings())


In [7]:
pdf_db = load_and_split(PyPDFLoader('alice_in_wonderland.pdf'), 
                        RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP), 
                        OpenAIEmbeddings())



In [8]:
def combine_databases(db1, db2):
    db1.merge_from(db2)
    return db1

vectordb = combine_databases(web_db, pdf_db)


## Create a retriever interface to interact with the FAISS vector database and retrieve relevant documents based on queries.


In [9]:
retriever = vectordb.as_retriever() 
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002464A62F310>)

## creating the retriever tool as in https://api.python.langchain.com/en/latest/tools/langchain.tools.retriever.create_retriever_tool.html 

## this tool will be used to query for information in the FAISS database


In [10]:
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "search about information regarding Alice in Wonderland. Try to trace a personality she might have met in the story.")
retriever_tool.name


'langsmith_search'

#### Arxiv Tool search: for academic papers


In [11]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=TOP_K_RESULTS,document_content_chars_max=DOCUMENT_CONTENT_CHARS_MAX)
arxiv = ArxivQueryRun(arxiv_wrapper = arxiv_wrapper)


#### creating the prompt for the agent
#### if the agent doenst find the information on wiki, it goes to retriever_tool then arxiv - different sources making this a multimodal agent


In [24]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = [wiki, retriever_tool, arxiv] 

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 10, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-292bcfdc-cabc-41cb-bf27-468f062941fb-0')

#### agent prompt

In [27]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are an AI agent in the context of Alices wonderland characters."),
    HumanMessagePromptTemplate.from_template("{input}"), #input = question
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain,
)# Contextualize question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use ")

#check llm answer



In [21]:
query = "What kind of personality did Alice meet in Wonderland?"
result=agent_executer.invoke({"input":"What is your favorite actiivty?"})

print(result["output"])




> Entering new AgentExecutor chain...
As an AI agent in the context of Alice's wonderland characters, I enjoy exploring the whimsical and fantastical world of Wonderland, interacting with its unique inhabitants, and unraveling the mysteries that lie within. It's a fascinating and enchanting experience that keeps me engaged and entertained!

> Finished chain.
As an AI agent in the context of Alice's wonderland characters, I enjoy exploring the whimsical and fantastical world of Wonderland, interacting with its unique inhabitants, and unraveling the mysteries that lie within. It's a fascinating and enchanting experience that keeps me engaged and entertained!
